## Integração de dados

#### Masterlens, CASTLES

In [1]:
import pandas as pd
from astropy.coordinates import Angle
import sqlite3
import os

In [3]:
# dec: -27:52:25.7 ---> -27.873806 deg
def sexdeg_to_deg(coord):
    result = Angle(coord + " degrees").degree
    return result #*3600


In [4]:
# ra: 00h49m41.89s ---> 12.424542 deg
def hour_to_deg(coord):  # string coord
    result = Angle(coord).degree
    return result #*3600

In [5]:
# ra: 00:49:41.89 ---> 00h49m41.89s
def hour(coord):
    result = coord.replace(":", 'h', 1)
    result = result.replace(":", 'm', 1)
    result += 's'
    return result

In [6]:
def name_clean(element):
    name = element.replace('+', '-')
    name = name.replace('.', '-')
    name = name.split('-')[0]
    
    if 'PMNJ' in name:
        name = name.replace('PMNJ', 'PMN')
        
    elif name == 'QJ0158':
        name = 'CTQ414'
        
    elif name == 'LBQS1009':
        name = name.replace('LBQS', 'Q')
    
    return name

In [6]:
# returns df with castles data (with coordinates converted to degrees)
def castles_df():
        # importing castles data  -  100 lenses
    castles = pd.read_csv('Scraping/castles.csv')

    castles['RA (J2000)'] = castles['RA (J2000)'].apply(hour)
    castles['RA (J2000)'] = castles['RA (J2000)'].apply(hour_to_deg)
    castles['Dec (J2000)'] = castles['Dec (J2000)'].apply(sexdeg_to_deg)
    castles['id'] = castles.index + 1 # from 1 to 100

    return castles

In [7]:
# returns df with castles data + imgs paths
def castles_imgs():
    
    castles_imgs = os.listdir('/home/viviane/GravLens/Database/Scraping/castles_imgs')
    names = []
    paths =[]
    for i in castles_imgs:
        name = i.rstrip('.gif')
        name = name.rstrip('H')
        name = name.rstrip('Hcc')
        name = name.rstrip('I')
        name = name.rstrip('Icc')
        name = name.rstrip('V')
        name = name.rstrip('Vcc')
        name = name.rstrip('K')
        name = name.rstrip('Kcc')
        name = name.rstrip('J')
        name = name.rstrip('Jcc')
        name = name.rstrip('R')
        name = name.rstrip('Rcc')
        
        names.append(name)
        path = i
        paths.append(path)

    images = {'name' : names, 'img_path' : paths }
    imgs_df = pd.DataFrame(images)
    
    # name - list of images
    imgs_df = imgs_df.groupby('name')['img_path'].apply(list)
    
    # castles df saved before in csv
    castles = castles_df()
    castles['name'] = castles['Lens Name'].apply(name_clean)
    castles.set_index(castles['name'], inplace=True)
    castles.drop('name', axis=1, inplace=True)
    
    # joining imgs df with system df
    result = pd.concat([castles, imgs_df], axis=1, join='outer', sort=False)
    result.set_index(castles['id'], inplace=True)
        
    return result

In [8]:
castles_imgs()

,Lens Name,zs,zl,RA (J2000),Dec (J2000),E(B-V),ms (mag),ml (mag),FGHz (mJy),Nim,"size ("")",dt (days),sigma (km/s),id,img_path
id,,,,,,,,,,,,,,,
1,Q0047-2808,3.60,0.48,12.424542,-27.873806,0.016,NaN,I=20.05,NaN,4ER,2.70,NaN,229±15,1,"[Q0047I.gif, Q0047Hcc.gif, Q0047V.gif, Q0047Ic..."
2,HE0047-1756,1.66,0.41,12.615958,-17.669111,0.022,I=16.53/2,I=18.97,NaN,2,1.44,NaN,NaN,2,"[HE0047I.gif, HE0047Icc.gif, HE0047V.gif, HE00..."
3,HST01247+0352,NaN,NaN,21.185000,3.866667,0.029,I=24.13/2,I=21.86,NaN,2,2.20,NaN,NaN,3,"[HST01247I.gif, HST01247Icc.gif, HST01247Hcc.g..."
4,HST01248+0351,NaN,NaN,21.190000,3.851667,0.029,NaN,NaN,NaN,2,0.74,NaN,NaN,4,NaN
5,B0128+437,3.124,NaN,22.805854,43.970317,0.082,NaN,NaN,F5=48,4,0.55,NaN,NaN,5,"[B0128Icc.gif, B0128I.gif]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,CY2201-3201,3.9,0.32,330.386667,-32.028889,0.028,NaN,NaN,NaN,2,0.83,NaN,130±20,96,NaN
97,HDFS2232509-603243,NaN,NaN,338.212083,-60.545278,0.028,NaN,NaN,NaN,4?E,0.90,NaN,NaN,97,NaN
98,Q2237+030,1.69,0.04,340.126417,3.358000,0.071,I=15.16/4,I=14.15,F5=0.336,4,1.78,NaN,215±30,98,"[Q2237Kcc.gif, Q2237H.gif, Q2237.gif, Q2237Vcc..."


In [9]:
# importing masterlens data  -  n(50) lenses
# returns df with masterlens data from tables: system, coordinates(deg) and image paths

def masterlens_df(n):
    
    ml_files_path = '/home/viviane/GravLens/Database/Scraping/masterlens_results'
    ml_imgfolders_path = '/home/viviane/GravLens/Database/Scraping/masterlens_imgs'
    
    dfs=[]
    for i in range(1, n+1):
        
    # getting data from system file      
        ml = pd.read_csv(ml_files_path + '/system_{}.csv'.format(i))

            # selecting important columns
        cols=[]
        for column in ['Name', 'Alternate Names', 'Discovery', 'Discovery Date', 'Lens Kind', 'Lens Grade',
                        'Description', 'N Images', 'Einstein_R', 'Einstein_R quality',
                        'Stellar_v_disp', 'Stellar_v_disp_err']:
            if column in list(ml.columns):
                cols.append(column)
        ml = ml[cols]


    # getting coords from coords file
        coords = pd.read_csv(ml_files_path + '/coordinates_{}.csv'.format(i))
        ml['Ra(deg)_ml'] = coords['RA [°]']
        ml['Dec(deg)_ml'] = coords['Dec [°]']
    
    
    # getting image paths
        system_folder = ml_imgfolders_path + '/masterlens_{}_arquivos'.format(i)
        imgs_list = os.listdir(system_folder)
        imgs = [system_folder + img for img in imgs_list]
        ml['img_path'] =  str(imgs)
        
        
        ml['id'] = i + len(castles_df())
        dfs.append(ml)
        
    all_ml = pd.concat(dfs, sort=False)
    all_ml.set_index(all_ml['id'], inplace=True)
        
    return all_ml

In [10]:
masterlens_df(50)

,Name,Discovery,Discovery Date,Lens Kind,Lens Grade,Description,N Images,Einstein_R,Einstein_R quality,Stellar_v_disp,Stellar_v_disp_err,Ra(deg)_ml,Dec(deg)_ml,img_path,id,Alternate Names
id,,,,,,,,,,,,,,,,
101,SDSS J0008-0004,SLACS,2008-08-01,GAL-GAL,A,Luminous red galaxy lensing three background o...,3,1.160,SIE model,232.0,39.0,2.012310,-0.068960,['/home/viviane/GravLens/Database/Scraping/mas...,101,NaN
102,ACS J001423.02-302109.8,Serendipitous,2012-06-15,GAL-GAL,B,Arc,1,1.520,SIE model,NaN,NaN,3.595917,-30.352722,['/home/viviane/GravLens/Database/Scraping/mas...,102,NaN
103,ACS J001426.26-302255.9,Serendipitous,2012-06-15,GAL-GAL,A,Arc,1,1.000,SIE model,NaN,NaN,3.609417,-30.382194,['/home/viviane/GravLens/Database/Scraping/mas...,103,NaN
104,CSWA 21,CASSOWARY,2007-06,GAL-GAL,A,This system is the brightest Lyman Break Gala...,4,2.910,SIE model,397.0,55.0,5.670492,14.519565,['/home/viviane/GravLens/Database/Scraping/mas...,104,8 O'Clock Arc
105,SDSS J0029-0055,SLACS,2008-08-01,GAL-GAL,A,Luminous red galaxy lensing two background so...,2,0.960,SIE model,202.0,18.0,7.282400,-0.930710,['/home/viviane/GravLens/Database/Scraping/mas...,105,NaN
106,Abell 68,Serendipitous,2007-06-01,XRAY-CLUST,A,NaN,8,7.500,Reference redshift,NaN,NaN,9.271704,9.163992,['/home/viviane/GravLens/Database/Scraping/mas...,106,A68
107,SDSS J0044+0113,SLACS,2008-08,GAL-GAL,A,Comments on IFU+HST Data: Clear coincidence of...,2,0.790,SIE model,254.0,12.0,11.012080,1.220180,['/home/viviane/GravLens/Database/Scraping/mas...,107,NaN
108,Q0047-2808,Serendipitous,1996-01-01,GAL-QSO,A,Bright Einstein ring-like feature. The source ...,4,1.340,NaN,219.0,12.0,12.424747,-27.874020,['/home/viviane/GravLens/Database/Scraping/mas...,108,ER 0047-2808
109,HE0047-1756,HE survey,2004-05-01,GAL-QSO,A,The newly discovered double QSO HE 0047−1756 ...,2,0.751,SIE model,NaN,NaN,12.615500,-17.668890,['/home/viviane/GravLens/Database/Scraping/mas...,109,NaN


## Lens as a whole
### Merge  - Name

In [9]:
# system table ---------->  143 lenses
def system():  # lens as a whole
    
    # importing castles data  -  100 lenses
    castles = pd.read_csv('Scraping/castles.csv')
    castles.drop(['zs', 'zl', 'ms (mag)', 'ml (mag)', 'dt (days)'], axis=1, inplace=True)
    castles['RA (J2000)'] = castles['RA (J2000)'].apply(hour)
    castles['RA (J2000)'] = castles['RA (J2000)'].apply(hour_to_deg)
    castles['Dec (J2000)'] = castles['Dec (J2000)'].apply(sexdeg_to_deg)
    castles.columns=['Name', 'RA(deg)_c', 'Dec(deg)_c', 'E(B-V)_c', 'FGHz (mJy)_c', 'Nimages_c',
                     'size(arccsec)_c', 'sigma (km/s)_c']
    #castles = castles.set_index('Name')
    castles['id_c'] = castles.index
    
    
    # importing masterlens data  -  50 lenses
    dfs = []
    for i in range(1,51):
        
        # getting data from system file
        ml = pd.read_csv('Scraping/masterlens_results/system_{}.csv'.format(i))

        # selecting important columns
        cols=[]
        for column in ['Name', 'Alternate Names', 'Discovery', 'Discovery Date', 'Lens Kind', 'Lens Grade',
                       'Description', 'N Images', 'Einstein_R', 'Einstein_R quality',
                       'Stellar_v_disp', 'Stellar_v_disp_err']:
            if column in list(ml.columns):
                cols.append(column)
        ml = ml[cols]

        # getting coords from coords file
        coords = pd.read_csv('Scraping/masterlens_results/coordinates_{}.csv'.format(i))
        ml['Ra(deg)_ml'] = coords['RA [°]']
        ml['Dec(deg)_ml'] = coords['Dec [°]']

        dfs.append(ml)
     
    # joining all masterlens system + coords dfs
    all_ml = pd.concat(dfs, sort=False)
    #all_ml = all_ml.set_index('Name')
    #all_ml['id_ml'] = range(100, 150)
    
    
    concat = pd.concat([castles, all_ml], sort=False)
    concat.reset_index(inplace=True, drop=True)
    concat.to_csv('concat.csv')
    # joining castles + ml data
    result = pd.merge(castles, all_ml, how ='outer', on='Name')  


    return result
        


### Merge - NACluster

In [12]:
# Preparing to run NACluster
def system_cluster():  # lens as a whole
    
    # importing castles data  -  100 lenses
    castles = castles_df()
    castles = castles[['RA (J2000)', 'Dec (J2000)' ]]
    castles.columns = ['ra', 'dec']
    
    castles['idCatalog'] = 1
   
    
    # importing masterlens data  -  50 lenses
    ml = masterlens_df()
    ml = ml[['Ra(deg)_ml', 'Dec(deg)_ml']]
    ml.columns = ['ra', 'dec']
    
    ml['idCatalog'] = 2
    
    # concatenating both catalogs to use Nacluster
    final = pd.concat([castles, ml])
    final.reset_index(drop = True, inplace=True)
    final = final.sample(frac=1)  # shuffling dataframe
    
    # saving csv file
    #system_na .to_csv('catalogs.csv', header=False)

    return final

In [13]:
system_na = system_cluster()
system_na

,ra,dec,idCatalog
112,32.372207,-6.719794,2
86,294.604958,66.814500,1
59,203.894958,1.301528,1
34,150.336583,55.897056,1
122,41.642076,-8.426707,2
...,...,...,...
43,157.305625,26.392167,1
105,9.271704,9.163992,2
44,158.392000,7.190417,1
69,216.158708,22.933500,1


In [14]:
# After running NACluster (Java)
# system table  ------------> 138 clustres
def NAC_result():
    old_file = open("clusters_catalogs.csv", 'r+')
    new_file = open("clusters.csv", 'a+')

    lastline=''
    cluster = 0
    header='id,ra,dec,cluster\n'
    new_file.write(header)
    for line in old_file:

        if 'cluster' in line:
            cluster+=1

        elif "Centroid" not in line:
            line = line.replace('(', '')
            line = line.replace(')', '')
            
            newline = ','.join(line.split(',')[0:3]) + ',{}'.format(cluster)
            new_file.write(newline)
            new_file.write('\n')
            
               
    old_file.close()
    new_file.close()
    
    return None
        

In [84]:
NAC_result()

In [89]:
def integrate():
    castle = castles_imgs()
    ml = masterlens_df(50)
    
    nac = pd.read_csv('clusters.csv')  # only has ra, dec, id, cluster
    nac = nac.sort_values(by=['id'])
    nac.reset_index(inplace=True, drop=True)
    
    # first concat nac with castle
    result = pd.concat([nac, castle.add_suffix('_c')], join ='outer', sort=True, axis=1)  #remove axis=1 to drop 'id' columns

    # then concat nac + castle with ml
    integrated = pd.concat([result, ml.add_suffix('_ml')], join ='outer', sort=True, axis=1)
    
    # combining dup rows into one   
    integrated = integrated.groupby('cluster').first()


    integrated = integrated.drop(['id_c', 'id_ml', 'ra', 'dec'], axis=1)
    integrated['isReal'] = 1  #  column to distinguish between real and simulated lenses (real=1, sim=0)
    integrated['Target'] = 1  #  column to distinguish between images with ot without lens (lens=1, nolens=0)

#    system.to_csv("integrated_system.csv", index=False)
    return integrated

In [90]:
integrate()

,id,Lens Name_c,zs_c,zl_c,RA (J2000)_c,Dec (J2000)_c,E(B-V)_c,ms (mag)_c,ml (mag)_c,FGHz (mJy)_c,...,Einstein_R_ml,Einstein_R quality_ml,Stellar_v_disp_ml,Stellar_v_disp_err_ml,Ra(deg)_ml_ml,Dec(deg)_ml_ml,img_path_ml,Alternate Names_ml,isReal,Target
cluster,,,,,,,,,,,,,,,,,,,,,
1.0,106.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,7.500,Reference redshift,NaN,NaN,9.271704,9.163992,['/home/viviane/GravLens/Database/Scraping/mas...,A68,1,1
2.0,11.0,SDSS0246-0825,1.68,NaN,41.642125,-8.426722,0.026,I=16.98/2,I=20.82,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1
3.0,123.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,SIE model,NaN,NaN,41.642076,-8.426707,['/home/viviane/GravLens/Database/Scraping/mas...,SDSS J024634.09-082536.1,1,1
4.0,139.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,137.862500,5.847780,['/home/viviane/GravLens/Database/Scraping/mas...,NaN,1,1
5.0,38.0,SDSS1004+4112,1.734,0.68,151.145458,41.211889,0.013,i=17.53/4,i=18.42,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1
6.0,59.0,SDSS1332+0347,1.445,0.191,203.094250,3.794417,0.026,i=18.70/2,i=18.64,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1
7.0,103.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.000,SIE model,NaN,NaN,3.609417,-30.382194,['/home/viviane/GravLens/Database/Scraping/mas...,NaN,1,1
8.0,3.0,HST01247+0352,NaN,NaN,21.185000,3.866667,0.029,I=24.13/2,I=21.86,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1
9.0,2.0,HE0047-1756,1.66,0.41,12.615958,-17.669111,0.022,I=16.53/2,I=18.97,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1


In [18]:
# Comparing merge by name with NAC

clusters = list(nac["cluster"])
dupes = list(set([x for x in clusters if clusters.count(x) > 1])) # cluster duplicates

clusters_dupes = nac.loc[nac['cluster'].isin(dupes)] 
#clusters_dupes

#merge_name = pd.read_csv('concat.csv')
#problem = [31, 141, 37, 149, 27, 135, 12, 125, 10, 122]
#merge_name.loc[merge_name.index.isin(problem)]

# Merge by name is wrong
# Use merge with NAC

NameError: name 'nac' is not defined

## Lens objects

In [19]:
# Scraping Joao
# lens objects  -  227
castles2 = pd.read_csv('CastelLensData.csv')
castles2 = castles2.set_index('lens_names')

HE_c = castles2.loc[ 'HE0435-1223' , : ]
HE_c

,ra,ra_err,dec,dec_err,description
lens_names,,,,,
HE0435-1223,4.63747,0.000,-12.2873,0.000,deflected
HE0435-1223,3.16147,0.003,-11.7343,0.003,deflected
HE0435-1223,2.17047,0.003,-12.8903,0.005,deflected
HE0435-1223,3.69847,0.003,-13.9013,0.003,deflected
HE0435-1223,3.47247,0.003,-12.8603,0.003,galaxy


In [21]:
# Object table
#HE_ml = pd.read_csv('Scraping/results/redshift_28.csv')
#HE_ml.drop('Unnamed: 1', axis=1, inplace=True)
#HE_ml.rename(columns={"Unnamed: 0": "Description"})

## BLF Challenge Simulations

In [7]:
# creating dataframe with blf simulations: catalog + img paths
def blf_train_df():
    #blf_train_path = '/home/viviane/BLF_training'
    blf_train_path = '/home/vivianemm/BLF'
    
    #blf = pd.read_csv(blf_train_path + '/traintest.csv', comment='#')
    blf = pd.read_csv(blf_train_path + '/BLF_catalog.csv', comment='#')
    
    ids = blf['ID']
    
    # Adding columns with paths for images in the 4 filters: img_path_H, img_path_J, img_path_VIS, img_path_Y
    for filter in ['H', 'J', 'VIS', 'Y']:
        path = blf_train_path + '/Images/EUC_{}/imageEUC_{}-'.format(filter, filter)
        complete_paths = [path + '{}.fits.png'.format(id) for id in ids]
        
        if
        blf['img_path_{}'.format(filter)] = complete_paths
        
    blf.to_csv('blf_complete.csv', index=False)
    
    return blf

In [2]:
os.listdir("/home/vivianemm/BLF/Images/EUC_H")

['imageEUC_H-200240.fits',
 'imageEUC_H-267605.fits.png',
 'imageEUC_H-242484.fits.png',
 'imageEUC_H-261055.fits',
 'imageEUC_H-269066.fits.png',
 'imageEUC_H-269787.fits.png',
 'imageEUC_H-220315.fits',
 'imageEUC_H-244666.fits.png',
 'imageEUC_H-279188.fits',
 'imageEUC_H-227819.fits.png',
 'imageEUC_H-276045.fits.png',
 'imageEUC_H-242407.fits.png',
 'imageEUC_H-269393.fits.png',
 'imageEUC_H-238222.fits.png',
 'imageEUC_H-222865.fits',
 'imageEUC_H-234759.fits.png',
 'imageEUC_H-232041.fits',
 'imageEUC_H-294321.fits.png',
 'imageEUC_H-243507.fits.png',
 'imageEUC_H-233991.fits.png',
 'imageEUC_H-205771.fits',
 'imageEUC_H-224867.fits.png',
 'imageEUC_H-267132.fits',
 'imageEUC_H-220064.fits.png',
 'imageEUC_H-253532.fits.png',
 'imageEUC_H-284105.fits',
 'imageEUC_H-269315.fits.png',
 'imageEUC_H-222213.fits.png',
 'imageEUC_H-269774.fits',
 'imageEUC_H-248216.fits',
 'imageEUC_H-239464.fits',
 'imageEUC_H-283714.fits',
 'imageEUC_H-204565.fits',
 'imageEUC_H-203814.fits.png',
 '

In [8]:
blf = blf_train_df()
len(blf)

100000

In [9]:
blf.head()

,ID,x_crit,y_crit,source_ID,z_source,z_lens,mag_source,ein_area,n_crit,r_source,...,n_gal_5,n_gal_10,halo_mass,star_mass,mag_lens,n_sources,img_path_H,img_path_J,img_path_VIS,img_path_Y
4719,200001,3.467590e-07,1.388970e-07,5131,1.96,0.699792,22.0469,3.714770e-10,19,0.000003,...,2,5,5.046820e+13,6.426010e+11,20.2312,1,/home/vivianemm/BLF/Images/EUC_H/imageEUC_H-20...,/home/vivianemm/BLF/Images/EUC_J/imageEUC_J-20...,/home/vivianemm/BLF/Images/EUC_VIS/imageEUC_VI...,/home/vivianemm/BLF/Images/EUC_Y/imageEUC_Y-20...
5721,200002,2.440650e-07,-2.128040e-07,7183,1.96,0.522781,25.4845,5.677310e-10,12,0.000003,...,0,4,1.027460e+14,8.192230e+11,18.9784,1,/home/vivianemm/BLF/Images/EUC_H/imageEUC_H-20...,/home/vivianemm/BLF/Images/EUC_J/imageEUC_J-20...,/home/vivianemm/BLF/Images/EUC_VIS/imageEUC_VI...,/home/vivianemm/BLF/Images/EUC_Y/imageEUC_Y-20...
3090,200003,4.086340e-07,2.556650e-07,4746,2.10,0.684567,25.4078,1.857780e-09,41,0.000017,...,2,10,4.105040e+14,1.151740e+12,19.2772,0,/home/vivianemm/BLF/Images/EUC_H/imageEUC_H-20...,/home/vivianemm/BLF/Images/EUC_J/imageEUC_J-20...,/home/vivianemm/BLF/Images/EUC_VIS/imageEUC_VI...,/home/vivianemm/BLF/Images/EUC_Y/imageEUC_Y-20...
4786,200004,7.584890e-08,-1.600640e-08,4426,1.93,0.540055,25.6997,7.307720e-10,24,0.000015,...,4,9,1.729910e+14,4.497190e+11,19.1146,1,/home/vivianemm/BLF/Images/EUC_H/imageEUC_H-20...,/home/vivianemm/BLF/Images/EUC_J/imageEUC_J-20...,/home/vivianemm/BLF/Images/EUC_VIS/imageEUC_VI...,/home/vivianemm/BLF/Images/EUC_Y/imageEUC_Y-20...
2874,200005,2.980050e-07,3.114750e-07,3389,2.62,0.734329,26.0315,6.982860e-10,23,0.000006,...,3,9,7.507740e+13,8.194420e+11,20.3108,1,/home/vivianemm/BLF/Images/EUC_H/imageEUC_H-20...,/home/vivianemm/BLF/Images/EUC_J/imageEUC_J-20...,/home/vivianemm/BLF/Images/EUC_VIS/imageEUC_VI...,/home/vivianemm/BLF/Images/EUC_Y/imageEUC_Y-20...


## Main code

In [158]:
# Preparing to run NACluster
system_prep = system_cluster()

# run NACluster (Java)
# Saves results in "clusters_catalogs.csv"

# After running NACluster:
# reads "clusters_catalogs.csv"
# Saves result in "clusters.csv"
NAC_result()

# Final system df after integration
# reads "clusters.csv"
# Saves result to "integrated_system.csv"
system_df = integrate()

